In [1]:
!which python

/opt/anaconda3/bin/python


In [1]:
from transformers import BertTokenizer, TFBertModel
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):
cannot import name 'maybe_sync' from 'fsspec.asyn' (/opt/anaconda3/lib/python3.7/site-packages/fsspec/asyn.py)

In [4]:
?DataCollatorForLanguageModeling

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = TFBertModel.from_pretrained("bert-base-cased")
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='tf')
output = model(encoded_input)

In [ ]:
#uploaded = files.upload()
#data_df = pd.read_csv(io.BytesIO(uploaded['msds543_chatbotdata.csv']))
#varieties_df = pd.read_csv(io.BytesIO(uploaded['wine_list.csv']))
#blogOsVinhos = pd.read_excel("BlogOsVinhos.xlsx")

data_df = pd.read_csv(URL + "msds543_chatbotdata.csv")
varieties_df = pd.read_csv(URL + "wine_list.csv")
blogOsVinhos = pd.read_excel(URL + "BlogOsVinhos.xlsx")
blogOsVinhos.rename(columns={"Name": "name", "Region": "region", "Producer": "brand"
                            }, inplace=True)

In [ ]:
def strip_accents(STR):
    return ''.join(c for c in unicodedata.normalize('NFD', STR)
                   if unicodedata.category(c) != 'Mn')

def clean_doc(doc): 
    """
    Given a string doc,
    this function cleans the doc
    of . . .
    non-ascii chars
    punctuation
    anything other than lower case words
    non-alphabetic tokens
    stop words
    tokens shorter than length 3
    stem, lemma are also options
    returns the list of tokens.
    """
    doc = re.sub(r'[^\x00-\x7F]', '', str(doc)) # removes any remaining non-ascii chars
    
    tokens = doc.split(" ") # set the tone
    
    # punctuation situation
    sub_punc = re.compile('[%s]' % re.escape(string.punctuation))
    tokens = [str(sub_punc.sub('', w)) for w in tokens]
    
    # alpha
    tokens = [word for word in tokens if word.isalpha()]
    
    # shorties
    tokens = [word for word in tokens if len(word) > 3]
    
    # capitalization
    tokens = [word.lower() for word in tokens]
    
    # stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    
    # word stemming    
    # ps=PorterStemmer()
    # tokens=[ps.stem(word) for word in tokens]
    
    # lemma
    return tokens # tokens is a list

In [ ]:
blogOsVinhos.head()

In [ ]:
blogOsVinhos['Castes'] = blogOsVinhos['Castes'].apply(clean_doc)
blogOsVinhos['processed_text'] = blogOsVinhos['JudgeNotes'].apply(clean_doc)

In [ ]:
blogOsVinhos.head()

In [ ]:
data_df.head()

In [ ]:
blogOsVinhos.iloc[0].processed_text

In [ ]:
blogOsVinhos = blogOsVinhos.explode('Castes')

In [ ]:
blogOsVinhos['variety'] = blogOsVinhos['Castes'] 

In [ ]:
varieties = [variety.lower() for variety in varieties_df.variety.unique()] + ["white", "red", "syrah"] + list(blogOsVinhos.variety.unique())
varieties = [item for item in varieties if not(pd.isnull(item)) == True]

In [ ]:
data_df['name_lower'] = data_df.name.apply(lambda xyz: xyz.lower())

In [ ]:
def get_variety(text):
  for variety in varieties:
    if variety.lower() in text:
      return variety

In [ ]:
data_df['variety'] = data_df.name_lower.apply(get_variety)

In [ ]:
#data_df = data_df.fillna('white')
data_df = data_df.fillna(value=np.nan)
data_df.dropna(subset=['variety'], inplace=True)

In [ ]:
data_df.head()

In [ ]:
data_df = data_df[['variety', 'processed_text']]

In [ ]:
blogOsVinhos = blogOsVinhos[['variety', 'processed_text']]

In [ ]:
#data_df = pd.concat([data_df, blogOsVinhos])
data_df = blogOsVinhos

In [ ]:
data_df.tail()

In [ ]:
## a peak
data_df['variety'][0:400].value_counts().plot.bar()
#data_df['variety'].value_counts().plot.bar()

In [ ]:
nan_value = float("NaN")
data_df.replace("", nan_value, inplace=True)
data_df.replace("None", nan_value, inplace=True)
data_df.dropna(subset=['variety'], inplace=True)
data_df.dropna(inplace=True)
#data_df.drop_duplicates(inplace=True)

In [ ]:
data_df.reset_index(inplace=True)
data_df.drop(columns=['index'], inplace=True)

In [ ]:
data_df.shape

In [ ]:
documents = data_df.processed_text.to_list()
string_labels = data_df.variety.to_list()

le = preprocessing.LabelEncoder()
le.fit(data_df.variety)
data_df['cat_variety'] = le.transform(data_df.variety)

labels = data_df.cat_variety.to_list()

In [ ]:
data_df.tail()

In [ ]:
#type(data_df.iloc[10907].processed_text)

In [ ]:
type(data_df.iloc[0].processed_text)

In [ ]:
train_texts, valid_texts, train_labels, valid_labels = train_test_split(documents, labels, test_size=0.33)
target_names = varieties

In [ ]:
# tokenize the dataset, truncate when passed `max_length`, 
# and pad with 0's when less than `max_length`

#train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=max_length)
#valid_encodings = tokenizer(valid_texts, truncation=True, padding=True, max_length=max_length)

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=max_length, is_split_into_words=True)
valid_encodings = tokenizer(valid_texts, truncation=True, padding=True, max_length=max_length, is_split_into_words=True)

In [ ]:
class WineGroupsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

# convert our tokenized data into a torch Dataset
train_dataset = WineGroupsDataset(train_encodings, train_labels)
valid_dataset = WineGroupsDataset(valid_encodings, valid_labels)

In [ ]:
# load the model and pass to CUDA
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(target_names)).to("cuda")

In [ ]:
from sklearn.metrics import accuracy_score

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  # calculate accuracy using sklearn's function
  acc = accuracy_score(labels, preds)
  return {
      'accuracy': acc,
  }

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=20,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
    # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
    logging_steps=200,               # log & save weights each logging_steps
    save_steps=400,
    evaluation_strategy="steps",     # evaluate each `logging_steps`
)

In [ ]:
trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=valid_dataset,          # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)

In [ ]:
# train the model
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
# saving the fine tuned model & tokenizer
model_path = "english-wine-bert-uncased" #"wine-bert-base-uncased"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

In [ ]:
def get_prediction(text, convert_to_label=False):
    # prepare our text into tokenized sequence
    inputs = tokenizer(text, padding=True, truncation=True, max_length=max_length, return_tensors="pt").to("cuda")
    # perform inference to our model
    outputs = model(**inputs)
    # get output probabilities by doing softmax
    probs = outputs[0].softmax(1)
    # executing argmax function to get the candidate label
    d = {v: k for v, k in enumerate(data_df.variety)}
    if convert_to_label:
      return d[int(probs.argmax())]
    else:
      return int(probs.argmax())

In [ ]:
input1a = """I would like a type of wine that really wows. something perhaps to drink beside my best girlfriends on the beach somewhere. something that's new and exciting."""

In [ ]:
get_prediction(input1a, convert_to_label=True)

In [ ]:
input1b = """Eu gostaria de um tipo de vinho que realmente impressionasse. algo talvez para beber ao lado de minhas melhores amigas na praia em algum lugar. algo que é novo e excitante.""" 

In [ ]:
get_prediction(input1b, convert_to_label=True)

In [ ]:
input2a = """
i want a red to go with my salmon. I like my salmon cooked medium and I like white wines normally but I hear reds are full of antioxidents.
"""

In [ ]:
get_prediction(input2a, convert_to_label=True)

In [ ]:
input2b = """
eu quero um vermelho para ir com o meu salmão. Eu gosto do meu salmão médio e gosto de vinhos brancos normalmente, mas ouvi dizer que os tintos são cheios de antioxidantes.
"""

In [ ]:
get_prediction(input2b, convert_to_label=True)

In [ ]:
input3a = """
tickles the nose
"""

In [ ]:
get_prediction(input3a, convert_to_label=True)

In [ ]:
input3b = """
faz cócegas no nariz
"""

In [ ]:
get_prediction(input3b, convert_to_label=True)

In [ ]:
input4a = """
I want the driest white wine available. the most dry you can think of. bone-dry.
"""

In [ ]:
get_prediction(input4a, convert_to_label=True)

In [ ]:
input4b = """
Quero o vinho branco mais seco disponível. o mais seco que você pode pensar. Osso seco.
"""

In [ ]:
get_prediction(input4b, convert_to_label=True)

In [ ]:
input5a = """
I want a red. no wait I want a white. I think I want rose. on second thought, give me a bourbon. no, no. my favorite wine is a white. I'll do a white?
"""

In [ ]:
get_prediction(input5a, convert_to_label=True)

In [ ]:
input5a = """
Eu quero um vermelho. não espere, eu quero um branco. Acho que quero rosa. pensando bem, me dê um bourbon. não não. meu vinho favorito é um branco. Eu vou fazer um branco?
"""

In [ ]:
get_prediction(input5a, convert_to_label=True)